# Part 0: imports & constants

In [43]:
import numpy as np
import music21 as m21
import pretty_midi as pm
import tempfile

test_midi_folder = 'midiFiles/'

# Part 1: file to list of PCVs 

## Part 1.1 code

In [8]:
def recursively_map_offset(midi_stream, only_note_name=True):
    '''
    This function will recursively walk through the Midi stream's elements, 
    and whenever it encounters a note, it will append its rhytmic 
    data to the pitch and then store the resulting data structure in an array.
    If a music21 element of type chord is encountered, the chord is decomposed into all
    notes it is composed by, and each of thoses notes are appended the 
    Returns the aforementionned array.
    The rhytmic data is expressed as a tuple of the offset of the beginning of the note
    and the offset of the end of the note.
    
    All temporal informations from MIDI events parsed by the music21 library are encoded
    as unit of quarter notes count, regardless of the bpm or the time signature.
    
    Params: 
    midi stream: the MIDI stream containing all the relevant infos
    flatten: Boolean, indicating whether or not the chords elements 
            need to be flattened into singles notes.
    only_note_name: Boolean, indicating whether the notes need to be 
                    converted from music21 object with octave indication
                    to only a string indicating the pitch.
    '''
    
    res = []
    for elem in midi_stream.recurse():
        if isinstance(elem, m21.note.Note):
            start_offset = elem.getOffsetInHierarchy(midi_stream)
            res.append((elem.name if only_note_name else elem, (start_offset, start_offset+elem.duration.quarterLength)))
        elif isinstance(elem, m21.chord.Chord):
            start_offset = elem.getOffsetInHierarchy(midi_stream)
            res += list(map(lambda r: (r.name if only_note_name else r , (start_offset, start_offset+elem.duration.quarterLength)), elem.pitches))
    return res


def remove_drums_from_midi_file(midi_filename):
    '''
    Takes care of removing drum tracks from a midi filename.
    Work only if the MIDI file has metadata clearly indicating channels that are
    percussive. Does not remove channels of percussive instruments that are pitched
    (like the glockenspiel for instance).  
    
    Param: 
    
    returns:
    The (temporary) filepath of the midi file generated without the drum channel.
    
    '''
    sound = pm.PrettyMIDI(midi_path)
    
    #getting
    drum_instruments_index = [idx for idx, instrument in enumerate(sound.instruments) if instrument.is_drum]
    for i in sorted(drum_instruments_index, reverse=True):
        del sound.instruments[i]

    folder = tempfile.TemporaryDirectory()
    temp_midi_filepath = folder.name+'tmp.mid'
    sound.write(temp_midi_filepath)
    
    return temp_midi_filepath


def only_keep_pitches_in_boundaries(pitch_offset_list, beat1_offset, beat2_offset): 
    return list(filter(lambda n: n[1][1] >= beat1_offset and n[1][0] <= beat2_offset, pitch_offset_list))


def slice_according_to_beat(pitch_offset_list, beat1_offset, beat2_offset):
    #the beat offset must be expressed as relation of quarter note. 
    #Taken are all beat which at least END AFTER the beat1, and START BEFORE the beat2
    res = []
    if beat1_offset >= beat2_offset:
        return res
    for n in only_keep_pitches_in_boundaries(pitch_offset_list, beat1_offset, beat2_offset):
        start_b = n[1][0]
        end_b = n[1][1]
        
        res_n = None
        if start_b >= beat1_offset:
            if end_b > beat2_offset:
                res_n = (n[0], (start_b, beat2_offset))
            else:
                res_n = (n[0], (start_b, end_b))
        elif end_b <= beat2_offset:
            #if start_b < beat1_offset: #of course we are in this case since the first if was not triggered.
            res_n = (n[0], (beat1_offset, end_b))
        else:
            #we are thus in the case the start and end time of the note overshoot the boundaries.
            res_n = (n[0], (beat1_offset, beat2_offset))
        #normally inconsistent results should not happen, but it is possible to have a note with duration equals to 0. This line below prevents that and thus keep the things concise.
        if res_n[1][0] < res_n[1][1]:
            res.append(res_n)
    return res

def sum_into_pitch_class_vector(pitch_offset_list, start_beat, end_beat):
    pitch_class_offset = lambda t: pitch_index_dict[normalize_notation_dict[t[0]]]
    pitch_class_vec = np.zeros(12)
    for tup in pitch_offset_list:
        #we need to be sure we don't take into account the part of the note that exceed the window's size.
        min_beat = max(start_beat, tup[1][0])
        max_beat = min(end_beat, tup[1][1])
        pitch_weight = max_beat - min_beat
        pitch_class_vec[pitch_class_offset(tup)] += pitch_weight
    return pitch_class_vec



def pitch_class_set_vector_from_pitch_offset_list(pitch_offset_array, chunk_number=0.0): #the analysis window size (aw_size) is expressed in terms of number of quarter.
    max_beat = get_max_beat(pitch_offset_array)
    chunk_number = chunk_number if chunk_number > 0. else max_beat
    aw_size = float(max_beat)/chunk_number
    res_vector = np.full((chunk_number, 12), 0.0, np.float32)

    for b in range(math.ceil(max_beat/aw_size)):
        start_beat = b*aw_size
        stop_beat = (b+1)*aw_size
        analysis_window = slice_according_to_beat(pitch_offset_array, start_beat, stop_beat)
        pitch_class_vec = sum_into_pitch_class_vector(analysis_window, start_beat, stop_beat)
        res_vector[b] = pitch_class_vec
    
    return res_vector


def produce_pitch_class_matrix_from_filename(filename, remove_percussions = True, aw_size = 0.5):
    '''
    TODO comments
    '''
    if filename.endswith('.mid') or filename.endswith('.midi'):
        midi_filename = remove_drums_from_midi_file(filename) if remove_percussions else filename
        pitch_offset_list = recursively_map_offset(midi_filename)
        
    elif filename.endswith('.wav'):
        return None
        #TODO: add code there.
    else:
        raise Exception('The file should be in MIDI or WAV format')
        
    return recursively_map_offset(midi_stream)

## Part 1.2 tests

In [22]:
midi_path = test_midi_folder+'NCandP.mid'